# Data Cleaning & Locating Multivalued and Duplicate records (single csv file)
### Saksham Gakhar, DA - DKSF

Keep changing the input csv file and look for duplicate and multivalued records, enlist devices that generally misbehave...

In [1079]:
import numpy as np 
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from collections import defaultdict
import datetime
# without mpld3
%matplotlib notebook 

In [1080]:
dates = pd.date_range('2017-09-01','2020-07-01', freq='MS').strftime("%Y-%m").tolist()

In [1081]:
dt = dates[0]
folder = '2020-07-06-DataKind/'
filename = 'output-' + str(dt) + '-01T00_00_00+00_00.csv'

In [1082]:
df = pd.read_csv(folder+filename)
df.when_captured = pd.to_datetime(df.when_captured)

Need to change the format of the Time Stamp for all the measurements in the raw data

In [1083]:
df.service_uploaded =  df.service_uploaded.apply(lambda x: datetime.datetime.strptime(x, '%b %d, %Y @ %H:%M:%S.%f')\
                                                 .replace(tzinfo=datetime.timezone.utc))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64655 entries, 0 to 64654
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   service_uploaded  64655 non-null  datetime64[ns, UTC]
 1   when_captured     61839 non-null  datetime64[ns, UTC]
 2   device_urn        64655 non-null  object             
 3   device_sn         64655 non-null  object             
 4   device            64655 non-null  int64              
 5   loc_lat           64655 non-null  float64            
 6   loc_lon           64655 non-null  float64            
 7   env_temp          36580 non-null  float64            
 8   env_humid         36580 non-null  float64            
 9   pms_pm01_0        36899 non-null  object             
 10  pms_pm02_5        36898 non-null  object             
 11  pms_pm10_0        36897 non-null  object             
 12  lnd_7318c         55230 non-null  float64            
 13  l

In [1084]:
df[0:5]

,service_uploaded,when_captured,device_urn,device_sn,device,loc_lat,loc_lon,env_temp,env_humid,pms_pm01_0,pms_pm02_5,pms_pm10_0,lnd_7318c,lnd_7318u,bat_voltage
0,2017-09-30 23:58:45+00:00,2017-09-30 23:58:38+00:00,safecast:374304606,Solarcast #30009,374304606,35.746,139.918,39.044,-7.584,NaN,NaN,NaN,32.0,29.0,NaN
1,2017-09-30 23:58:28+00:00,2017-09-30 23:58:23+00:00,safecast:374304606,Solarcast #30009,374304606,35.746,139.918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-09-30 23:57:41+00:00,2017-09-30 23:57:30+00:00,safecast:374304606,Solarcast #30009,374304606,35.746,139.918,NaN,NaN,11,14,14,NaN,NaN,NaN
3,2017-09-30 23:53:51+00:00,2017-09-30 23:53:50+00:00,safecast:3714913954,Solarcast #30027,3714913954,52.102,5.180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.795
4,2017-09-30 23:53:39+00:00,2017-09-30 23:53:35+00:00,safecast:3714913954,Solarcast #30027,3714913954,52.102,5.180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Based on above table for (`device`, `when_captured`) key, let's see what these multiple values for each time stamp correspond to. Sometimes there are negative RH, sometimes 0.0 PM (which measn very clean air)

In [1085]:
def findBadData(df):
    
    temp_df = df.groupby(['device_urn', 'device_sn','when_captured']).size().to_frame('size').\
                                    reset_index().sort_values('size', ascending=False)
    print("bad device data counts: ")
    print(temp_df[(temp_df['size']>1)])
    
    print("all bad device list: ")
    # Devices that have misbehaved at some point - more than one data values per time stamp
    print(np.unique(temp_df[temp_df['size']>1]['device_sn'].values)) # devices that have misbehaved

In [1086]:
findBadData(df)

bad device data counts: 
                device_urn         device_sn             when_captured  size
5634    safecast:114699387  Solarcast #30023 2017-09-11 08:30:04+00:00     5
5639    safecast:114699387  Solarcast #30023 2017-09-11 14:45:03+00:00     5
5635    safecast:114699387  Solarcast #30023 2017-09-11 09:45:04+00:00     5
5636    safecast:114699387  Solarcast #30023 2017-09-11 11:00:03+00:00     5
5637    safecast:114699387  Solarcast #30023 2017-09-11 12:15:03+00:00     5
5638    safecast:114699387  Solarcast #30023 2017-09-11 13:30:03+00:00     5
5633    safecast:114699387  Solarcast #30023 2017-09-11 06:00:04+00:00     5
5641    safecast:114699387  Solarcast #30023 2017-09-11 16:00:03+00:00     3
31971  safecast:3714913954  Solarcast #30027 2017-09-29 09:22:50+00:00     2
32368  safecast:3714913954  Solarcast #30027 2017-09-30 13:52:50+00:00     2
32417  safecast:3714913954  Solarcast #30027 2017-09-30 17:22:50+00:00     2
31495  safecast:3714913954  Solarcast #30027 2017-0

#### Add a column for the year

In [1087]:
df['year'] = pd.DatetimeIndex(df['when_captured']).year

Enlist some anomalies with same key (when, devSN) but different data fields

## Data Cleansing based on [Protocol](https://github.com/DataKind-SF/safecast/blob/master/Solarcast_data_cleansing.md)

In [1088]:
print(df['when_captured'].isna().sum())
df = df[df['when_captured'].notna()]

df.shape

2816


(61839, 16)

In [1089]:
boolean_condition = df['when_captured'] >  pd.to_datetime(2000/1/19, infer_datetime_format=True).tz_localize('UTC')
print(boolean_condition.sum())
df = df[df['when_captured'] >  pd.to_datetime(2000/1/19, infer_datetime_format=True).tz_localize('UTC')]

df.shape

61839


(61839, 16)

In [1090]:
boolean_condition = (df['env_humid']<0) | (df['env_humid']>100)
print(boolean_condition.sum())
column_name = 'env_humid'
new_value = np.nan
df.loc[boolean_condition, column_name] = new_value

df.shape

9102


(61839, 16)

In [1091]:
boolean_condition = abs(df['when_captured'].subtract(df['service_uploaded'])).astype('timedelta64[D]') < 7
boolean_condition.shape
print("to remove: ",df.shape[0] - (boolean_condition).sum())
df = df[boolean_condition]

df.shape

to remove:  0


(61839, 16)

Dont need the following column ANY MORE

In [1092]:
df.drop('service_uploaded', axis=1, inplace=True)
df.shape

(61839, 15)

Drop Duplicates

In [1093]:
incoming = df.shape[0]
df.drop_duplicates(subset=df.columns[0:df.shape[1]], inplace=True, keep='first') # args: subset=[df.columns[0:df.shape[1]]], keep = 'first'
print(-df.shape[0]+incoming)

8


### Filtering bad row records

In [1094]:
temp_df = df.groupby(['device_sn','when_captured']).agg(['count','nunique'])
# temp_df.info()
num_groups = temp_df.shape[0]
print(num_groups)

61801


Merge Counts and Count-Distincts to check for duplicative records and multiplicities

In [1095]:
even = list(range(0,26,2))
odd = list(range(1,26,2))
tmp_df1 = temp_df.iloc[:,even].max(axis=1).to_frame('COUNTS').reset_index()
tmp_df2 = temp_df.iloc[:,odd].max(axis=1).to_frame('DISTINCTS').reset_index()
print(tmp_df1.shape, tmp_df2.shape)
merged = pd.merge(tmp_df1, tmp_df2, left_on = ['device_sn', 'when_captured'], right_on=['device_sn', 'when_captured'])
merged

(61801, 3) (61801, 3)


,device_sn,when_captured,COUNTS,DISTINCTS
0,Solarcast #30001,2017-09-07 19:58:47+00:00,1,1
1,Solarcast #30001,2017-09-07 19:58:59+00:00,1,1
2,Solarcast #30001,2017-09-07 19:59:14+00:00,1,1
3,Solarcast #30001,2017-09-07 19:59:29+00:00,1,1
4,Solarcast #30001,2017-09-07 19:59:44+00:00,1,1
...,...,...,...,...
61796,Solarcast #30030,2017-09-30 23:17:09+00:00,1,1
61797,Solarcast #30030,2017-09-30 23:32:09+00:00,1,1
61798,Solarcast #30030,2017-09-30 23:32:33+00:00,1,1
61799,Solarcast #30030,2017-09-30 23:32:48+00:00,1,1


#### Calculating hits: Impose mutually exclusive conditions for filtering

Actionable: Records of useless data with all NaNs

In [1096]:
bool1 = (merged.COUNTS >1) & (merged.DISTINCTS==0)
sum1 = bool1.sum()
print(sum1)
toDiscard1 = merged.loc[:,['device_sn', 'when_captured']][bool1]
toDiscard1.shape

0


(0, 2)

Actionable: Records that are a mix of duplicates and non-duplicate rows for a given (`device_sn`, `when_captured`) [must be all discarded]

In [1097]:
bool3 = (merged.COUNTS >1) & (merged.DISTINCTS>1)
sum3 = bool3.sum()
print(sum3)
toDiscard3 = merged.loc[:,['device_sn', 'when_captured']][bool3]
toDiscard3.shape

8


(8, 2)

NOT Actionable as duplicates were dropped: Records where all rows are purely duplicates [preserve only 1 later]

In [1098]:
bool2 = (merged.COUNTS >1) & (merged.DISTINCTS==1)
sum2 = bool2.sum()
print(sum2)
print("get rid of : " ,merged.COUNTS[bool2].sum() - merged.DISTINCTS[bool2].sum())

0
get rid of :  0


Records that are good

In [1099]:
bool4 = (merged.COUNTS ==1) & (merged.DISTINCTS==1)
sum4 = bool4.sum()
print(sum4)

61793


In [1100]:
#ensure you have all records covered by 1 of the 4 conditions
assert(num_groups == sum1+sum2+sum3+sum4)

#### Filter now from the main dataframe


In [1101]:
discard = pd.concat([toDiscard1, toDiscard3], ignore_index=True)
discard['KEY_DevSN_WhenCapt'] = list(zip(discard.device_sn, discard.when_captured))
print(df.shape, discard.shape)

(61831, 15) (8, 3)


In [1102]:
df['KEY_DevSN_WhenCapt'] = list(zip(df.device_sn, df.when_captured))
df.shape

(61831, 16)

In [1103]:
rows_to_discard = df['KEY_DevSN_WhenCapt'].isin(discard['KEY_DevSN_WhenCapt'])
print("these many rows to discard: ", rows_to_discard.sum())

these many rows to discard:  38


In [1104]:
incoming = df.shape[0]
df = df[~rows_to_discard]
print(incoming - df.shape[0])

38


### Now check to make sure no garbage data is left

In [1105]:
findBadData(df)

bad device data counts: 
Empty DataFrame
Columns: [device_urn, device_sn, when_captured, size]
Index: []
all bad device list: 
[]


In [1106]:
toDiscard3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 40839 to 40847
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   device_sn      8 non-null      object             
 1   when_captured  8 non-null      datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), object(1)
memory usage: 192.0+ bytes


In [1107]:
toDiscard3.to_csv(str(dt) + '-01_anomalies.csv')